#### Tkinter/ coupang

In [24]:
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
import tkinter as tk
from selenium import webdriver
from selenium.webdriver.common.by import By
import subprocess
import chromedriver_autoinstaller

from PIL import Image, ImageTk
from urllib.request import urlopen
from io import BytesIO
import webbrowser

from Class_1_Searching_blogs import BlogManager
from Class_2_Searching_coupang import Setting_Options
from Class_3_Shopping_coupang import ShopManager
from Class_5_Searching_review import ReviewManager


chrome_path = (
    r'C:\Program Files\Google\Chrome\Application\chrome.exe '
    r'--remote-debugging-port=9221 --user-data-dir="C:\chrometemp" '
    r'--window-size=400,300 --window-position=0,0'
)
chrome_process = subprocess.Popen(chrome_path)

options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9221")

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=options)

index_num = 0

display4 = None
scrollbar_display4 = None

coupang_rev = ReviewManager(driver)
coupang = Setting_Options(driver)
coupang_list = ShopManager(driver)

def delete_and_insert():
    display1.config(state=NORMAL)
    display2.config(state=NORMAL)
    display3.config(state=NORMAL)
   
    display1.delete(1.0, END)
    display2.delete(1.0, END)
    display3.delete(1.0, END)
  
    display1.insert(1.0, title_list[index_num])
    display2.insert(1.0, price_list[index_num])
    display3.insert(1.0, point_list[index_num])

    display1.config(state=DISABLED)
    display2.config(state=DISABLED)
    display3.config(state=DISABLED)

def blog_delete_and_insert():
    global blog_title_list, blog_contents_list, blog_username, blog_index_num
    
    ## 텍스트에 삽입하거나 삭제하기 
    display5.delete(1.0, END)
    display6.delete(1.0, END)
    display7.delete(1.0, END)

    display5.insert(1.0, "[블로그 제목]\n")
    display6.insert(1.0, "[블로그 내용]\n")
    display7.insert(1.0, "[블로그 주인장]\n")

    display5.insert(2.0, blog_title_list[blog_index_num])
    display6.insert(2.0, blog_contents_list[blog_index_num])
    display7.insert(2.0, blog_username[blog_index_num])


def open_link():
    webbrowser.open(link_list[index_num])

def img_insert():
    image_url = imageLink_list[index_num]
    with urlopen(image_url) as response:
        image_data = response.read()
        image = Image.open(BytesIO(image_data))
        photo = ImageTk.PhotoImage(image)
        label1.configure(image=photo)
        label1.image = photo

def keyword_submit():
    keyword_got = keyword_entry.get()
    information_list.config(text=f"<{keyword_got}의 쇼핑 목록>")


def search_Blog():
    global blog_title_list, blog_contents_list, blog_username, blog_num

    blog_title_list=[]
    blog_contents_list=[]
    blog_username=[]
    
    messagebox.showinfo("크롤링 진행 중", "블로그 검색 결과를 가져오는 중입니다. 잠시 기다려주세요...")

    chromedriver_autoinstaller.install()
    driver = webdriver.Chrome()

    blog_search_data=BlogManager(driver)
    blog_search_data.Search_and_Scroll(keyword_entry2.get())
    blog_username,  blog_title_list, blog_contents_list  =blog_search_data.find_html()  
        
    blog_keyword_submit()
    blog_command_current()


def blog_keyword_submit():
    ## 어떤 블로그 목록인지 
    keyword_got2 = keyword_entry2.get()
    keyword_got2 = f"<{keyword_got2}에 대한 블로그>"
    blog_information_list.configure(text=keyword_got2)

def command_current():
    global index_num
    index_num = 0
    img_insert()
    delete_and_insert()

def blog_command_current():
    global blog_index_num

    blog_index_num=0

    blog_delete_and_insert()

def clickNext():
    global index_num

    index_num += 1
    if index_num > len(imageLink_list) - 1:
        index_num=0

    img_insert()
    delete_and_insert()
    display4_forget()

def clickPrev():
    global index_num

    index_num -= 1
    if index_num < 0:
        index_num = len(imageLink_list) - 1

    img_insert()
    delete_and_insert()
    display4_forget()

def BlogclickNext():
    global blog_index_num, blog_username
        
    
    if blog_index_num>len(blog_username)-1:
        blog_index_num=0
    else:
        blog_index_num+=1

    blog_delete_and_insert()

def BlogclickPrev():
    global blog_index_num, blog_username

    if blog_index_num<0:
        blog_index_num = len(blog_username)-1
    else:
        blog_index_num-=1

    blog_delete_and_insert()

def display4_forget():
    if display4:
        display4.grid_forget()
    if scrollbar_display4:
        scrollbar_display4.grid_forget()



def search_action():
    global link_list, imageLink_list, title_list, price_list, point_list

    messagebox.showinfo("크롤링 진행 중", "검색 결과를 가져오는 중입니다. 잠시 기다려주세요...")

    search_query = keyword_entry.get()

    
    try:
        xpath = "//div[@class='main-today__img-container c1-image']"
        driver.find_element(By.XPATH, xpath)
    except:
        pass
    else:
        coupang.getting_keyword(search_query)

    try:
        xpath="//div[@class='prod-image__items']"
        driver.find_element(By.XPATH, xpath)
    except:
        pass
    else:
        driver.get("http://coupang.com/")
        coupang.getting_keyword(search_query)

    try:
        xpath = "//div[contains(@class, 'search-filter-options search-')]"
        driver.find_elements(By.XPATH, xpath)
    except:
        pass
    else:
        link_list, imageLink_list, title_list, price_list, point_list = coupang_list.find_html()
        keyword_submit()
        keyword_entry.delete(0, END)
        
        command_current()
        grid_data()
        display4_forget()



def review_link():
    global display4, scrollbar_display4

    messagebox.showinfo("크롤링 진행 중", "리뷰를 가져오는 중입니다. 잠시 기다려주세요...")

    re5 = coupang_rev.Loading_reviews(link_list[index_num], 5)
    
    scrollbar_display4 = Scrollbar(scrollable_frame)
    display4 = Text(scrollable_frame, width=80, height=30, borderwidth=5, yscrollcommand=scrollbar_display4.set)
    scrollbar_display4.config(command=display4.yview)
    
    display4.grid(column=0, row=9, columnspan=4, pady=5)
    scrollbar_display4.grid(column=4, row=9, sticky='ns')
    
    display4.config(state=DISABLED)
    display4.config(state=NORMAL)
    display4.delete(1.0, END)
    display4.insert(1.0, re5)
    display4.config(state=DISABLED)

def _on_mouse_wheel(event):
    canvas.yview_scroll(int(-1*(event.delta/120)), "units")

def Options_list():
    search_query = keyword_entry.get()
    
    try:
        xpath = "//div[@class='main-today__img-container c1-image']"
        driver.find_element(By.XPATH, xpath)
    except:
        pass
    else:
        coupang.getting_keyword(search_query)

    try:
        xpath="//div[@class='prod-image__items']"
        driver.find_element(By.XPATH, xpath)
    except:
        pass
    else:
        driver.get("http://coupang.com/")
        coupang.getting_keyword(search_query)

    try:
        xpath = "//div[contains(@class, 'search-filter-options search-')]"
        driver.find_elements(By.XPATH, xpath)
    except:
        pass
    else:
        option_names = coupang.print_Option_names()
    
    return option_names

def option_selected(options, opt=None):
    coupang.Click_option(options, opt)
    
def update_options():
    messagebox.showinfo("크롤링 진행 중", "옵션을 가져오는 중입니다. 잠시 기다려주세요...")
    

    options = Options_list()
    option_menu = OptionMenu(scrollable_frame, var, *options, 
                             command=lambda opt=options: (option_selected(options=opt), update_option_items()))
    option_menu.grid(column=0, row=3)
    

def update_option_items():
    options = coupang.print_Option_items(var.get())
    option_menu2 = OptionMenu(scrollable_frame, var2, *options, 
                              command=lambda opt=options: option_selected(options=var.get(), opt=opt))
    option_menu2.grid(column=1, row=3)

def grid_data():
    information_list.grid(column=1, row=4, pady=5)
    label1.grid(column=1, row=5, pady=5)
    display1.grid(column=0, row=6, columnspan=4, pady=5)
    display2.grid(column=0, row=7, columnspan=4, pady=5)
    display3.grid(column=0, row=8, columnspan=4, pady=5)
    
    link_button.grid(column=0, row=10, pady=5)
    reset_button.grid(column=1, row=10, pady=5)
    rev_button.grid(column=3, row=10, pady=5)

    btnPrev.grid(column=0, row=11, pady=5)
    btnNext.grid(column=3, row=11, pady=5)
    
    instructions.grid(column=0, row=12, columnspan=4, pady=5)
    
def reset():
    global link_list, imageLink_list, title_list, price_list, point_list
    
    driver.get("https://coupang.com/")
    messagebox.showinfo("리셋 완료", "리셋되었습니다. 다시 검색해주세요.")

    link_list, imageLink_list, title_list, price_list, point_list = [], [], [], [], []
    
    display1.grid_forget()
    display2.grid_forget()
    display3.grid_forget()
    display4_forget()

def show_window():
    selected_tab = tab.select()
    selected_tab_index=tab.index(selected_tab)
    print(f"Tab changed to {selected_tab_index + 1}")

    if selected_tab_index ==0:
        print("첫번째 탭 입력됨")
    
    elif selected_tab_index ==1:
        print("두번째 탭 입렫됨")



driver.get("http://coupang.com/")
window=Tk()
window.title("Coupang Search")
window.geometry("600x800")
window.resizable(False, False)
tab=ttk.Notebook(window)
tab.pack(expand=True, fill='both')
tab1 = ttk.Frame(tab)
tab.add(tab1, text="쿠팡 검색")

canvas = Canvas(tab1)
scrollbar = Scrollbar(tab1, orient=VERTICAL, command=canvas.yview)
scrollable_frame = Frame(canvas)

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(
        scrollregion=canvas.bbox("all")
    )
)
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

canvas.pack(side=LEFT, fill=BOTH, expand=True)
scrollbar.pack(side=RIGHT, fill=Y)

information0 = Label(scrollable_frame, text="<쿠팡 쇼핑 시스템>\n")
information1 = Label(scrollable_frame, text="검색어 입력")
information2 = Label(scrollable_frame, text="페이지 숫자")
information3 = Label(scrollable_frame, text='옵션 선택')
information4 = Label(scrollable_frame, text='제품 검색')

keyword_entry = Entry(scrollable_frame)
page_num_entry = Entry(scrollable_frame)

select_options = Button(scrollable_frame, text="옵션", command=update_options)
search_button = Button(scrollable_frame, text="검색", command=search_action)
reset_button = Button(scrollable_frame, text="리셋", command=reset)

btnPrev = Button(scrollable_frame, text="<<이전", command=clickPrev)
btnNext = Button(scrollable_frame, text="다음>>", command=clickNext)

information_list = Label(scrollable_frame, text="<검색어의 쇼핑 목록>")
label1 = Label(scrollable_frame)

display1 = Text(scrollable_frame, width=80, height=3, borderwidth=5)
display2 = Text(scrollable_frame, width=80, height=3, borderwidth=5)
display3 = Text(scrollable_frame, width=80, height=3, borderwidth=5)

display1.config(state=DISABLED)
display2.config(state=DISABLED)
display3.config(state=DISABLED)

link_button = Button(scrollable_frame, text="Open Link", command=open_link)
rev_button = Button(scrollable_frame, text="review link", command=review_link)
instructions = Label(scrollable_frame,
    text=(
        "<사용 설명서>\n\n"
        "첫 번째 빈 항목에 검색어를 입력하고\n"
        "두 번째 빈 항목에 원하는 페이지를 입력하세요\n\n"
        "한 페이지당 보통 50개~70개 정도 검색 가능하며\n"
        "그 후 검색 버튼을 누르면 페이지마다 쇼핑 목록을 볼 수 있습니다\n"
    )
)
var = StringVar(value="선택없음")
var2 = StringVar(value="선택없음")

canvas.bind_all("<MouseWheel>", _on_mouse_wheel)
canvas.bind_all("<Button-4>", _on_mouse_wheel)
canvas.bind_all("<Button-5>", _on_mouse_wheel)

information0.grid(row=0, column=0, columnspan=4, pady=5)
information1.grid(row=1, column=0)
information2.grid(row=1, column=1)
information3.grid(row=1, column=2)
information4.grid(row=1, column=3)
keyword_entry.grid(row=2, column=0)
page_num_entry.grid(row=2, column=1)
select_options.grid(row=2, column=2)
search_button.grid(row=2, column=3)

tab2 = ttk.Frame(tab,)
tab.add(tab2, text="블로그 검색")

information5 = Label(tab2, text=f"<블로그 탐색>\n")         
information5.grid(column=4, row=0)

keyword_entry2 = Entry(tab2)
keyword_entry2.grid(column=4, row=2)

search_button2 = Button(tab2, text="검색", command=search_Blog)
search_button2.grid(column=6, row=2, columnspan=2)

BlogbtnPrev=Button(tab2, text="<<이전", command=BlogclickNext)   
BlogbtnNext=Button(tab2, text="다음>>", command=BlogclickPrev)

blog_information_list=Label(tab2, text="<검색어의 블로그>")
blog_information_list.grid(column=4, row=3)

display5 = Text(tab2, width=80, height=6, borderwidth=5)
display7 = Text(tab2, width=80, height=6, borderwidth=5)
display6 = Text(tab2, width=80, height=6, borderwidth=5)

display5.grid(column=4, row=5, columnspan=4, pady=10)
display7.grid(column=4, row=8, columnspan=4, pady=10)
display6.grid(column=4, row=11, columnspan=4, pady=10)

BlogbtnPrev.grid(column=5, row=20, pady=10)
BlogbtnNext.grid(column=6, row=20, pady=10)



tab.mainloop()
chrome_process.terminate()